## Image Generation

In [ ]:
!pip install transformers==4.49 diffusers==0.32.2

In [ ]:
!wget https://raw.githubusercontent.com/thiagohersan/media-landscape/refs/heads/main/python/utils.py
!wget https://raw.githubusercontent.com/thiagohersan/media-landscape/refs/heads/main/python/imgs/landscape_00.jpg

In [ ]:
import os
os.environ["GEMINI_API_KEY"] = ""
os.environ["NEWSDATA_API_KEY"] = ""

import json

from PIL import Image as PImage

from utils import get_articles, get_articles_with_top_words
from utils import get_img, get_img_description
from utils import Generator, get_pipeline, get_input_images

In [ ]:
# "runwayml/stable-diffusion-inpainting",
# "stable-diffusion-v1-5/stable-diffusion-inpainting",
# "stabilityai/stable-diffusion-2-inpainting",

pipe = get_pipeline("stable-diffusion-v1-5/stable-diffusion-inpainting")

In [ ]:
img00 = PImage.open("./landscape_00.jpg")
img, mask = get_input_images(img00, keep_width=256, size=(1440, 512))
# display(img)
# display(mask)

In [ ]:
prompt = get_img_description(img00)
anthropoprompt = f"post apocalyptic version of {prompt}, but with things on fire, floods and trash everywhere"
anthropoprompt

In [ ]:
generator = Generator(device="cuda").manual_seed(1010)

output = pipe(
  prompt=anthropoprompt,
  negative_prompt="repetitive, distortion, glitch, borders, stretched, frames, breaks, multiple rows",
  image=img,
  mask_image=mask,
  width=img.size[0], height=img.size[1],
  guidance_scale=16.0,
  num_inference_steps=24,
  generator=generator
)

display(output.images[0])


## Newsdata.io

In [ ]:
res = get_articles(n_articles=200)

with open(f"./data/newsdata_top_200.json", "w") as ofp:
  json.dump(res, ofp)

In [ ]:
q = "catastrophe"
res = get_articles(q=q, cat="environment", n_articles=200)

with open(f"./data/newsdata_{q}_200.json", "w") as ofp:
  json.dump(res, ofp)

In [ ]:
q = "rain"
res = get_articles(q=q, cat="environment", n_articles=200)

with open(f"./data/newsdata_{q}_200.json", "w") as ofp:
  json.dump(res, ofp)

In [ ]:
with open("./data/newsdata_catastrophe_200.json", "r") as ifp:
  newsdata_res = json.load(ifp)

art_idxs = get_articles_with_top_words(newsdata_res, n_words=20, n_articles=3)

art_idxs

In [ ]:
imgs = [get_img(newsdata_res[idx]["image_url"]) for idx in art_idxs.values.reshape(-1)]

imgs = [x for x in imgs if type(x) != str]

imgs_by_size = sorted(imgs, key=lambda x: x.size[0]*x.size[1], reverse=True)

for img in imgs_by_size:
  display(img)